# Ensembles: Random Forest, Bagging, Blending

**Исполнители (ФИО):** Your answer here

---

Здравствуйте! На прошедших семинарах мы разобрали все популярные идеи отдельных методов классического машинного обучение, однако, помимо этого есть ещё два подхода работы с этимим моделями, которые позволяют улучшить результаты в решении задач классификации и регрессии. На этом семинаре вы познакомитесь с первым из них - построение ансамблей

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Идея ансамблевого алгоритма заключается в использовании отдельных моделей для построения новой объединённой модели, основной принцип тут: Много разных моделей лучше одной. При таком подходе модели, входящие в ансамбль, начинают специализироваться на отдельных частях данных, что позволяет объединённой модели «запоминать» больше особенностей 

Есть два подхода построения ансамблей: 
1) Бэггинг -  объединение большого количества одинаковых моделей, каждая из которых обучается на случайной подвыборке из фичей и данных. Например, на данном принципе основан Случайный Лес (*Random Forest*), который по сути является объединением деревьев
2) Блендинг - объединение разных моделей, каждая из которых обучается на всей тренировочной выборке

## Задача 1

Загрузите датасет *TrafficViolation.csv*, в качестве таргета используйте колонку *Violation.Type*

In [ ]:
#Your code here

Используйте [Дерево решений](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) для классификации, осуществите подбор гиперпараметров с помощью [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Постройте полученное дерево, оцените качество классификации 

In [ ]:
#Your code here

Постройте [Случайный лес](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), используйте полученные параметры оптимального дерева. Оцените качество классификации

In [ ]:
#Your code here

Помимо всех уже известных вам параметров Дерева решений, Случайный лес имеет параметр *n_estimators* - количество деревьев в лесе. Он также определяет склонность леса к переобучению

Используйте [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html), чтобы подобрать оптимальное количество деревьев в построенном вами ранее лесе. Постройте зависимость метрики качества от параметра на тренировочной и валидационной выборках с помощью атрибутов класса GridSearchCV

In [ ]:
#Your code here

**Вопрос:** Как меняется качество классификации от количества деревьев на тренировочной и валидационной выборках? Какое оптимальное значение количества деревьев вы выбрали? Почему?

*Your answer here*

Лес, как и дерево, считает важность фичей. Выведите график важности фичей у вашего леса и постройте модель на основе важных фичей  

In [ ]:
#Your code here

Сравните все три модели: Дерево, Лес и Лес на важных фичах

In [ ]:
#Your code here

**Вопрос:** Какая модель оказалась лучшей? Почему?

*Your answer here*

## Задача 2

Перебор гиперпараметров по сетке хотя и является хорошим, но не оптимальным методом подбора наилучших параметров. Поиск по сетке можно улучшить с помощью [Байесовской оптимизации](https://ru.wikipedia.org/wiki/%D0%9E%D0%BF%D1%82%D0%B8%D0%BC%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D0%B3%D0%B8%D0%BF%D0%B5%D1%80%D0%BF%D0%B0%D1%80%D0%B0%D0%BC%D0%B5%D1%82%D1%80%D0%BE%D0%B2), которая позволяет проходить не по всей сетке, а на некотором оптимальном подможестве

Этот подход реализован в библиотеке [optuna](https://optuna.org/), ниже приведет код использования для упрощения работы с данной либой

In [ ]:
!pip install -q optuna

from optuna import create_study
from optuna.pruners import HyperbandPruner
from optuna.visualization import plot_optimization_history
from optuna.storages import RDBStorage
from optuna.trial import TrialState

def random_forest_opt(trial):
    # выберите диапозон рассматриваемых параметров
    params = {
        "criterion": trial.suggest_categorical("criterion", ['gini', 'entropy']),
        "max_depth": trial.suggest_int("max_depth", 2, 20, step=1),
        "random_state": 42,
    }

    clf = RandomForestClassifier(**params)
    clf.fit(X, y)

    return clf.score(X, y)

user_name = "postgres"
user_password = "postgres"
host = "127.0.0.1"
port = "5432"
database = "gyrostab_optuna" 
iterations = 1000 # выберите количество итераций алгоритма
hw_iter = 0
study_name = f"hw_{hw_iter}" # необобходимо менять для разных прогонов optuna
hw_iter += 1

url_str = f"postgresql+psycopg2://{user_name}:{user_password}@{host}:{port}/{database}"
storage = "sqlite:///optuna.db"

study = create_study(
    load_if_exists=True,
    storage=storage,
    study_name=study_name,
    pruner=HyperbandPruner,
    direction="maximize", # В зависимости от вашей метрики, выберите направление работы алгоритма
)

study.optimize(random_forest_opt, n_trials=iterations)

Загрузите свой датасет из проекта

In [ ]:
#Your code here

Используйте Случайный лес для [классификации](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) и [регрессии](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) на ваших данных. Подберите гиперпараметры используя [оптуну](https://optuna.readthedocs.io/en/stable/). Изобразите кривую обучения с помощью библиотеки [plotly](https://habr.com/ru/articles/502958/), подсветите на ней точку с оптимальными гиперпараметрами

In [ ]:
#Your code here

## Задача 3

Когда у вас уже есть какая-то хорошая модель, Бэггинг - хороший способ попробовать повысить качество, построив ансамбль на её основе. Как вы поните, его можно делать на базе любой модели, лес лишь частный случай. Обычно в Бэггинг объединяют десятки/сотни моделей, тем самым повышая шанс покрыть всю специфику данных, однако тут тоже нужно следить за переобучением 

Загрузите датасет *richters_piece_2.csv*

In [ ]:
#Your code here

Используйте [Случайный лес](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) для классификации, подберите гиперпараметры, оцените качетсво

In [ ]:
#Your code here

Теперь создайте лес самостоятельно. Для этого используйте [Бэггинг](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html) для создания классификатора на основе [Дерева решений](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html). Задайте параметры, аналогичные лесу выше

In [ ]:
#Your code here

Сравните Случайный лес и «Мешок деревьев»

In [ ]:
#Your code here

**Вопрос:** Какой из методов оказался лучше? Предположите почему? 

*Your answer here*

Теперь выберите любой другой известный вам метод (кроме дерева и леса). Постройте [Бэггинг](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html) на его оснвое. Подберите гиперпараметры у исходного метода и его Бэггинга

In [ ]:
#Your code here

Сравните три полученные модели

In [ ]:
#Your code here

**Вопрос:** Какие модели у вас получились? Какая из них оказалась лучшей? Предположите почему?

*Your answer here*

## Задача 4

Теперь разберёмся подробнее с Блендингом. Как вы помните, в отличии от Бэггинга, Блендинг это объединение разных моделей, для каждой из них нужно производить подбор гиперпараметров, потому, Блендинг имеет смысл использовать когда у вас есть несколько обученных моделей, каждая из которых хорошо работает на какой-то части данных, например, каждая из моделей умеет идеально предсказывать один из классов, таким образом, в Блендинге модели будут дополнять друг друга

Загрузите датасет *richters_whole_2.csv*

In [ ]:
#Your code here

Есть два типа блендинга в зависимости от того, как принимается итоговое решение на основе ответов отдельных моделей. Первый это VotingClassifier, по сути взвешанное голосование: каждая модель возвращает вероятности принадлежности классам, после чего общие вероятности пренадлежности классам вычисляются как взвешенная сумма, где веса - веса моделей в Блендинге (их нужно подбирать) 

Используйте две или более разных моделей классификации. Подберите гиперпараметры, оцените качество классификации, произведе диагностику моделей, используя ROC кривую

In [ ]:
#Your code here

Объедините ваши модели с помощью [VotingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html), произведите подбор весов в голосовании, оцените качество классификации и сравните с исходными моделями

In [ ]:
#Your code here

Второй вариант Блендинга это StackingClassifier: вероятности принадлежности классам от отдельных моделей идут на вход мета-модели (классификатор), которая принимает решение 

Теперь объедините ваши модели с помощью [StackingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html), оцените качество классификации и сравните между собой варианты с разным конечным классификатором

In [ ]:
#Your code here

Сравните лучший StackingClassifier с исходными моделями и VotingClassifier

In [ ]:
#Your code here

**Вопрос:** Какие модели вы взяли? Какая модель оказалось лучшей? Получилось ли улучшить качество с помощью Блендинга? Предположите почему?

*Your answer here*